In [1]:
#pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [6]:
import cv2
import numpy as np
from keras.models import load_model

model = load_model('my_model.h5')

In [7]:
def preprocess_image(image):
    resized_image = cv2.resize(image, (28, 28))
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    normalized_image = grayscale_image / 255.0
    reshaped_image = normalized_image.reshape(1, 28, 28, 1)
    return reshaped_image

cap = cv2.VideoCapture(0, cv2.CAP_AVFOUNDATION)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture frame")
        break
    
    processed_frame = preprocess_image(frame)
    
    prediction = model.predict(processed_frame)
    predicted_letter = chr(prediction.argmax() + 65)
    
    # Display prediction
    cv2.putText(frame, predicted_letter, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)
    
    cv2.imshow('ASL Prediction', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 25ms/step


2024-04-17 11:11:51.004 Python[41134:1993600] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


1/1 [==============================] - 0s 13ms/step
